In [1]:
import pandas as pd
from sqlalchemy import create_engine
from pathlib import Path
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "notebook_connected"



# Replace with your actual database connection info
engine = create_engine("sqlite:///trails.db")


# Load CSV
database_path = Path("../Resources/Trails.csv")
df = pd.read_csv(database_path)

df.to_sql("trails", engine, if_exists="replace", index=False)
df.columns = [col.replace("OSMPTrailsOSMP", "") for col in df.columns]

df.head()

,OBJECTID,OWNER,BICYCLES,DISPLAY,TRAILTYPE,DATEFROM,DATETO,SEGMENTID,HORSES,RID,...,OSMPTrailClosuresCLOSUREDURATION,OSMPTrailClosuresWEBLINK,OSMPTrailClosuresCLOSUREAREA,OSMPTrailClosuresTRAILSTATUS,OSMPTrailClosuresCLOSUREREASON,OSMPTrailClosuresLOCATIONDESCRIPTION,OSMPTrailClosuresCONTACT,OSMPTrailClosuresCOMMENTS,OSMPTrailClosuresGLOBALID,OSMPTrailClosuresSEGMENTID
0,2669,OSMP,No,Yes,Hiking Trail,2022/06/05 00:00:00+00,2099/12/31 00:00:00+00,289-334-328,Yes,2052,...,NaN,NaN,NaN,Open,Hazardous Conditions,NaN,https://bouldercolorado.gov/services/osmp-clos...,NaN,{3FFDA1D3-AF1A-46C7-87E2-4A4707D27043},289-334-328
1,2671,OSMP,Yes,Yes,Multi-Use Trail,2023/05/22 00:00:00+00,2099/12/31 00:00:00+00,518-678-538,Yes,2214,...,Until Further Notice,NaN,NaN,Open,Construction,NaN,https://bouldercolorado.gov/services/osmp-clos...,NaN,{13A86ADE-5F3C-42DF-9BB4-2068A290C8E4},518-678-538
2,2672,OSMP,Yes,Yes,Multi-Use Trail,2023/05/21 00:00:00+00,2099/12/31 00:00:00+00,506-550-678,Yes,2212,...,NaN,NaN,NaN,Open,NaN,NaN,https://bouldercolorado.gov/services/osmp-clos...,NaN,{AF6F3A35-0B7B-40B5-97B4-7E32A5F17791},506-550-678
3,2673,OSMP,Yes,Yes,Multi-Use Trail,2023/05/21 00:00:00+00,2099/12/31 00:00:00+00,506-678-551,Yes,2213,...,NaN,NaN,NaN,Open,NaN,NaN,https://bouldercolorado.gov/services/osmp-clos...,NaN,{E20E1606-73D2-4AA7-B6CD-1D7A3222D9D0},506-678-551
4,2675,OSMP,No,Yes,Hiking Trail,2021/07/28 00:00:00+00,2099/12/31 00:00:00+00,228-147-247,Yes,1979,...,NaN,NaN,NaN,Open,Maintenance,NaN,https://bouldercolorado.gov/services/osmp-clos...,Closed for NCAR Fire,{2BE6F52A-EA3B-434B-B169-CEC9C646747B},228-147-247


In [2]:
print(df.columns.tolist())


['OBJECTID', 'OWNER', 'BICYCLES', 'DISPLAY', 'TRAILTYPE', 'DATEFROM', 'DATETO', 'SEGMENTID', 'HORSES', 'RID', 'TRLID', 'MILEAGE', 'MEASUREDFEET', 'TRAILNAME', 'GlobalID', 'DIFFICULTY', 'DOGS', 'DOGREGGEN', 'DOGREGDESC', 'EBIKES', 'SHAPESTLength', 'OSMPTrailClosuresOBJECTID', 'OSMPTrailClosuresRID', 'OSMPTrailClosuresCLOSUREDURATION', 'OSMPTrailClosuresWEBLINK', 'OSMPTrailClosuresCLOSUREAREA', 'OSMPTrailClosuresTRAILSTATUS', 'OSMPTrailClosuresCLOSUREREASON', 'OSMPTrailClosuresLOCATIONDESCRIPTION', 'OSMPTrailClosuresCONTACT', 'OSMPTrailClosuresCOMMENTS', 'OSMPTrailClosuresGLOBALID', 'OSMPTrailClosuresSEGMENTID']


In [3]:
print(df['DOGS'].value_counts(dropna=False))
print(df['DOGREGDESC'].value_counts(dropna=False))

DOGS
Yes    569
No      29
Name: count, dtype: int64
DOGREGDESC
Leash, Or Voice and Sight Control     359
Leash Required                        179
No Dogs                                29
On-Corridor Voice and Sight            20
Leash Required on Portion of Trail      2
LVS M-F, Leash Required Sat & Sun       2
NaN                                     2
Leash, Or Voice and SIght Control       1
Leash, Or Voice and Sigth Control       1
Leash, Or Voice and Sight Contorl       1
Leash Required on South Portion         1
Regulation Varies                       1
Name: count, dtype: int64


In [4]:
# create categories for 
def categorize_dog_policy(desc):
    if pd.isna(desc):
        return None

    desc = desc.lower().strip()

    if "no dogs" in desc:
        return "no_dogs"
    elif "voice and sight" in desc or "off-leash" in desc or "on-corridor voice and sight" in desc:
        return "off_leash"
    elif "leash" in desc:
        return "leash_required"
    else:
        return None


In [5]:
df['dog_access'] = df['DOGREGDESC'].apply(categorize_dog_policy)

# Filter out any rows that don't match one of the 3
df = df[df['dog_access'].notnull()]

# Check the breakdown
print(df['dog_access'].value_counts())

dog_access
off_leash         381
leash_required    185
no_dogs            29
Name: count, dtype: int64


In [6]:
# Create separate data frames for each category

off_leash_df = df[df['dog_access'] == 'off_leash']
leash_required_df = df[df['dog_access'] == 'leash_required']
no_dogs_df = df[df['dog_access'] == 'no_dogs']


In [7]:
# check counts 
print(f"Off-leash trails: {len(off_leash_df)}")
print(f"Leash required trails: {len(leash_required_df)}")
print(f"No dogs allowed trails: {len(no_dogs_df)}")


Off-leash trails: 381
Leash required trails: 185
No dogs allowed trails: 29


In [8]:
dog_policy_dfs = {
    'off_leash': off_leash_df,
    'leash_required': leash_required_df,
    'no_dogs': no_dogs_df
}


In [9]:
# load and inspect JSON
import json
with open("../../OSMP_Trails.geojson", "r") as f:
    geojson = json.load(f)

# Check what it looks like
import pprint
pprint.pprint(geojson, depth=2)

{'crs': {'properties': {...}, 'type': 'name'},
 'features': [{...},
              {...},
              {...},
              {...},
              {...},
              {...},
              {...},
              {...},
              {...},
              {...},
              {...},
              {...},
              {...},
              {...},
              {...},
              {...},
              {...},
              {...},
              {...},
              {...},
              {...},
              {...},
              {...},
              {...},
              {...},
              {...},
              {...},
              {...},
              {...},
              {...},
              {...},
              {...},
              {...},
              {...},
              {...},
              {...},
              {...},
              {...},
              {...},
              {...},
              {...},
              {...},
              {...},
              {...},
              {...},
        

In [10]:
coords_list = []

for feature in geojson["features"]:
    props = feature["properties"]
    trail_id = props.get("OSMPTrailsOSMPTRLID")
    trail_name = props.get("OSMPTrailsOSMPTRAILNAME")
    
    coordinates = feature["geometry"].get("coordinates", [])
    if coordinates and isinstance(coordinates[0], list):
        lon, lat = coordinates[0]
        coords_list.append({
            "OSMPTrailsOSMPTRLID": trail_id,
            "OSMPTrailsOSMPTRAILNAME": trail_name,
            "latitude": lat,
            "longitude": lon
        })

coords_df = pd.DataFrame(coords_list)

In [11]:
# check column names in both DataFrames
print("Cleaned trail df columns:\n", df.columns.tolist())
print("Coordinates df columns:\n", coords_df.columns.tolist())

Cleaned trail df columns:
 ['OBJECTID', 'OWNER', 'BICYCLES', 'DISPLAY', 'TRAILTYPE', 'DATEFROM', 'DATETO', 'SEGMENTID', 'HORSES', 'RID', 'TRLID', 'MILEAGE', 'MEASUREDFEET', 'TRAILNAME', 'GlobalID', 'DIFFICULTY', 'DOGS', 'DOGREGGEN', 'DOGREGDESC', 'EBIKES', 'SHAPESTLength', 'OSMPTrailClosuresOBJECTID', 'OSMPTrailClosuresRID', 'OSMPTrailClosuresCLOSUREDURATION', 'OSMPTrailClosuresWEBLINK', 'OSMPTrailClosuresCLOSUREAREA', 'OSMPTrailClosuresTRAILSTATUS', 'OSMPTrailClosuresCLOSUREREASON', 'OSMPTrailClosuresLOCATIONDESCRIPTION', 'OSMPTrailClosuresCONTACT', 'OSMPTrailClosuresCOMMENTS', 'OSMPTrailClosuresGLOBALID', 'OSMPTrailClosuresSEGMENTID', 'dog_access']
Coordinates df columns:
 ['OSMPTrailsOSMPTRLID', 'OSMPTrailsOSMPTRAILNAME', 'latitude', 'longitude']


In [12]:
#rename object ID column in DF so it matches column name in coords_df
cols = df.columns.tolist()

# Find all the indexes of columns with the same name
dup_indexes = [i for i, col in enumerate(cols) if col == "OSMPTrailsOSMPTRLID"]

# Rename just the first one
if dup_indexes:
    cols[dup_indexes[0]] = "OBJECTID"  # or whatever name you want
    df.columns = cols
df.head()

,OBJECTID,OWNER,BICYCLES,DISPLAY,TRAILTYPE,DATEFROM,DATETO,SEGMENTID,HORSES,RID,...,OSMPTrailClosuresWEBLINK,OSMPTrailClosuresCLOSUREAREA,OSMPTrailClosuresTRAILSTATUS,OSMPTrailClosuresCLOSUREREASON,OSMPTrailClosuresLOCATIONDESCRIPTION,OSMPTrailClosuresCONTACT,OSMPTrailClosuresCOMMENTS,OSMPTrailClosuresGLOBALID,OSMPTrailClosuresSEGMENTID,dog_access
0,2669,OSMP,No,Yes,Hiking Trail,2022/06/05 00:00:00+00,2099/12/31 00:00:00+00,289-334-328,Yes,2052,...,NaN,NaN,Open,Hazardous Conditions,NaN,https://bouldercolorado.gov/services/osmp-clos...,NaN,{3FFDA1D3-AF1A-46C7-87E2-4A4707D27043},289-334-328,off_leash
1,2671,OSMP,Yes,Yes,Multi-Use Trail,2023/05/22 00:00:00+00,2099/12/31 00:00:00+00,518-678-538,Yes,2214,...,NaN,NaN,Open,Construction,NaN,https://bouldercolorado.gov/services/osmp-clos...,NaN,{13A86ADE-5F3C-42DF-9BB4-2068A290C8E4},518-678-538,leash_required
2,2672,OSMP,Yes,Yes,Multi-Use Trail,2023/05/21 00:00:00+00,2099/12/31 00:00:00+00,506-550-678,Yes,2212,...,NaN,NaN,Open,NaN,NaN,https://bouldercolorado.gov/services/osmp-clos...,NaN,{AF6F3A35-0B7B-40B5-97B4-7E32A5F17791},506-550-678,leash_required
3,2673,OSMP,Yes,Yes,Multi-Use Trail,2023/05/21 00:00:00+00,2099/12/31 00:00:00+00,506-678-551,Yes,2213,...,NaN,NaN,Open,NaN,NaN,https://bouldercolorado.gov/services/osmp-clos...,NaN,{E20E1606-73D2-4AA7-B6CD-1D7A3222D9D0},506-678-551,leash_required
4,2675,OSMP,No,Yes,Hiking Trail,2021/07/28 00:00:00+00,2099/12/31 00:00:00+00,228-147-247,Yes,1979,...,NaN,NaN,Open,Maintenance,NaN,https://bouldercolorado.gov/services/osmp-clos...,Closed for NCAR Fire,{2BE6F52A-EA3B-434B-B169-CEC9C646747B},228-147-247,off_leash


In [13]:
print(df.columns.tolist())

['OBJECTID', 'OWNER', 'BICYCLES', 'DISPLAY', 'TRAILTYPE', 'DATEFROM', 'DATETO', 'SEGMENTID', 'HORSES', 'RID', 'TRLID', 'MILEAGE', 'MEASUREDFEET', 'TRAILNAME', 'GlobalID', 'DIFFICULTY', 'DOGS', 'DOGREGGEN', 'DOGREGDESC', 'EBIKES', 'SHAPESTLength', 'OSMPTrailClosuresOBJECTID', 'OSMPTrailClosuresRID', 'OSMPTrailClosuresCLOSUREDURATION', 'OSMPTrailClosuresWEBLINK', 'OSMPTrailClosuresCLOSUREAREA', 'OSMPTrailClosuresTRAILSTATUS', 'OSMPTrailClosuresCLOSUREREASON', 'OSMPTrailClosuresLOCATIONDESCRIPTION', 'OSMPTrailClosuresCONTACT', 'OSMPTrailClosuresCOMMENTS', 'OSMPTrailClosuresGLOBALID', 'OSMPTrailClosuresSEGMENTID', 'dog_access']


In [14]:
df = df.rename(columns={"TRLID": "OSMPTrailsOSMPTRLID"})

In [15]:
print(df.loc[:, df.columns == "OSMPTrailsOSMPTRLID"].head())

   OSMPTrailsOSMPTRLID
0                  289
1                  518
2                  506
3                  506
4                  228


In [16]:
# Merge With Your Cleaned Trail Data
# Merge using the TRLID key
merged_df = df.merge(coords_df, on="OSMPTrailsOSMPTRLID", how="inner")

# Check results
print(merged_df[['OSMPTrailsOSMPTRAILNAME', 'dog_access', 'latitude', 'longitude']].head())


  OSMPTrailsOSMPTRAILNAME      dog_access   latitude   longitude
0           Mount Sanitas       off_leash  40.034241 -105.305444
1           Mount Sanitas       off_leash  40.020623 -105.296273
2           Mount Sanitas       off_leash  40.021561 -105.296570
3           Mount Sanitas       off_leash  40.020986 -105.296258
4           IBM Connector  leash_required  40.077186 -105.197753


In [17]:
merged_df.to_csv("merged_trails.csv", index=False)


In [18]:
# Visualize trals on a map

fig = px.scatter_mapbox(
    merged_df,
    lat="latitude",
    lon="longitude",
    color="dog_access",
    hover_name="OSMPTrailsOSMPTRAILNAME",  # or whatever name column you'd prefer
    zoom=11,
    height=600,
    mapbox_style="open-street-map"
)

fig.show()


In [19]:
# Base map setup
mapbox_args = dict(
    lat=merged_df['latitude'],
    lon=merged_df['longitude'],
    zoom=11,
    mapbox_style="open-street-map"
)

# Create one trace per group
off_leash = merged_df[merged_df['dog_access'] == 'off_leash']
leash_required = merged_df[merged_df['dog_access'] == 'leash_required']
no_dogs = merged_df[merged_df['dog_access'] == 'no_dogs']

fig = go.Figure()

fig.add_trace(go.Scattermapbox(
    lat=off_leash['latitude'],
    lon=off_leash['longitude'],
    mode='markers',
    marker=dict(size=8, color='green'),
    name='Off-Leash',
    text=off_leash['OSMPTrailsOSMPTRAILNAME'],
    visible=True
))

fig.add_trace(go.Scattermapbox(
    lat=leash_required['latitude'],
    lon=leash_required['longitude'],
    mode='markers',
    marker=dict(size=8, color='orange'),
    name='Leash Required',
    text=leash_required['OSMPTrailsOSMPTRAILNAME'],
    visible=False
))

fig.add_trace(go.Scattermapbox(
    lat=no_dogs['latitude'],
    lon=no_dogs['longitude'],
    mode='markers',
    marker=dict(size=8, color='red'),
    name='No Dogs',
    text=no_dogs['OSMPTrailsOSMPTRAILNAME'],
    visible=False
))

In [20]:
# Dropdown menu
fig.update_layout(
    updatemenus=[
        dict(
            buttons=list([
                dict(label="Off-Leash",
                     method="update",
                     args=[{"visible": [True, False, False]},
                           {"title": "Off-Leash Trails"}]),
                dict(label="Leash Required",
                     method="update",
                     args=[{"visible": [False, True, False]},
                           {"title": "Leash Required Trails"}]),
                dict(label="No Dogs",
                     method="update",
                     args=[{"visible": [False, False, True]},
                           {"title": "No Dogs Allowed Trails"}]),
                dict(label="Show All",
                     method="update",
                     args=[{"visible": [True, True, True]},
                           {"title": "All Trails"}])
            ]),
            direction="down",
            showactive=True,
            x=0.1,
            xanchor="left",
            y=1,
            yanchor="top"
        )
    ]
)

fig.update_layout(
    title="Boulder Trails: Dog Access Map",
    mapbox=dict(
        style="open-street-map"
        # no center/zoom here so it auto-fits
    ),
    margin={"r":0,"t":40,"l":0,"b":0}
)

fig.show()

In [21]:
import plotly.express as px
import pandas as pd

# Create summary DataFrame (grouped by dog access type)
dog_summary = merged_df['dog_access'].value_counts().reset_index()
dog_summary.columns = ['Dog Access', 'Count']

# Create bar chart with custom colors
fig_bar = px.bar(
    dog_summary,
    x="Dog Access",
    y="Count",
    title="🐶 Trail Counts by Dog Access Type",
    labels={"Dog Access": "Dog Policy", "Count": "Number of Trails"},
    color="Dog Access",
    color_discrete_map={
        "off_leash": "#add19e",       # soft green
        "leash_required": "#e29d12",  # warm mustard
        "no_dogs": "#dad6a1"          # muted tan
    },
    template="plotly_white"
)

# Layout and styling
fig_bar.update_layout(
    title_font_size=22,
    title_x=0.5,
    xaxis_title="Dog Policy",
    yaxis_title="Trail Count",
    xaxis=dict(tickfont=dict(size=14)),
    yaxis=dict(tickfont=dict(size=14)),
    legend_title="Dog Policy",
    bargap=0.3
)

fig_bar.show()

In [22]:
print(merged_df['MILEAGE'].describe())


count    4951.000000
mean        0.166714
std         0.263970
min         0.003000
25%         0.031000
50%         0.071000
75%         0.198000
max         3.692000
Name: MILEAGE, dtype: float64


In [23]:
import pandas as pd
import plotly.express as px

# Drop missing or zero-mileage values if needed
mileage_data = merged_df['MILEAGE'].dropna()
mileage_data = mileage_data[mileage_data > 0]

# Bin mileage into quantiles
mileage_data_qcut, bin_edges = pd.qcut(
    mileage_data, q=20, retbins=True, precision=2, duplicates='drop'
)

# Calculate midpoints and counts
bin_counts = mileage_data_qcut.value_counts(sort=False)
bin_midpoints = 0.5 * (bin_edges[1:] + bin_edges[:-1])

# Build new DataFrame
mileage_qdist = pd.DataFrame({
    "Mileage Bin (mi)": bin_midpoints,
    "Trail Count": bin_counts.values
})

# Create line chart
fig_line = px.line(
    mileage_qdist,
    x="Mileage Bin (mi)",
    y="Trail Count",
    title="Trail Mileage Distribution",
    markers=True,
    labels={
        "Mileage Bin (mi)": "Trail Mileage (mi)",
        "Trail Count": "Number of Trails"
    },
    line_shape="spline",
    template="plotly_white"
)

# Customize layout and appearance
fig_line.update_layout(
    title_font_size=22,
    title_x=0.5,
    width=700,  # narrower width
    height=400,
    xaxis=dict(
        title_font=dict(size=14),
        range=[0, 1]  # adjust this range as needed
    ),
    yaxis=dict(
        title_font=dict(size=14)
    )
)

fig_line.update_traces(
    marker=dict(size=6, color="#6BA368"),
    line=dict(color="#6BA368")
)

fig_line.show()




In [24]:
import plotly.express as px

df['dog_access'] = df['DOGREGDESC'].apply(categorize_dog_policy)


In [25]:
df_filtered = df[df['dog_access'].notnull()]
dog_summary = df_filtered['dog_access'].value_counts().reset_index()
dog_summary.columns = ['Dog Access', 'Count']

In [26]:
dog_summary['Dog Access'] = dog_summary['Dog Access'].replace({
    'off_leash': 'Off-Leash Allowed',
    'leash_required': 'Leash Required',
    'no_dogs': 'No Dogs'
})

In [27]:
print(dog_summary)

          Dog Access  Count
0  Off-Leash Allowed    381
1     Leash Required    185
2            No Dogs     29


In [28]:

# Create the pie chart with custom color mapping
fig = px.pie(
    dog_summary,
    names='Dog Access',
    values='Count',
    title='Boulder, CO Trails & Dog Access',
    color='Dog Access',
    color_discrete_map={
        'Off-Leash Allowed': '#add19e',
        'Leash Required': '#e29d12',
        'No Dogs': '#ada6a1'
    }
)
fig.update_traces(textinfo='label+percent')
fig.show()
